In [1]:
import pandas as pd
import regex as re
from tqdm import tqdm
import geograpy4
import numpy as np

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\hayadi\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\hayadi\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\hayadi\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     C:\Users\hayadi\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hayadi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hayadi\AppData\Roaming\nltk_data...
[nltk_data]   Package

In [2]:
sampled_no_retweet_df = pd.read_csv("sampled_no_retweet_df.csv", index_col=0)

In [3]:
locations = sampled_no_retweet_df[['id', 'loc']]

In [4]:
import json
# Opening JSON file
f = open('location_replacement_dict.json')
# returns JSON object as a dictionary
replace_dict = json.load(f)

In [5]:
locations = locations.dropna(subset=["loc"])
locations = locations.reset_index().drop(["index"],axis=1)

#Replace tweets fake locations by empty string and contractions by full location
all_keys = list(replace_dict.keys())
all_keys.sort(key=len)
all_keys = all_keys[::-1]
all_keys_lower = [x.lower() for x in all_keys]
all_results = [replace_dict[x] for x in all_keys]

In [6]:
def correct_location(locations):
    
    new_loc_str = re.sub("{", "" , locations)
    new_loc_str = re.sub("}", "" , new_loc_str)
    loc_lower = new_loc_str.lower()
    for i, key in enumerate(all_keys_lower):
        try:
            p=re.compile(r"\b"+key+r"\b")
            m = p.search(loc_lower)
            while m is not None :
                span = m.span()
                new_loc_str = new_loc_str[:span[0]]+ \
                        "{"+str(i)+"}" + \
                        new_loc_str[span[1]:]
                loc_lower = new_loc_str.lower()
                m = p.search(loc_lower)
            
            
        except AttributeError:
            continue
    
    new_loc_str = new_loc_str.format(*all_results)
    return new_loc_str
    

In [9]:
locations['correct_location'] = locations['loc'].apply(lambda x: correct_location(x))

KeyboardInterrupt: 

In [6]:
#locations.to_csv("non_retweeted_located_df.csv")
locations = pd.read_csv("non_retweeted_located_df.csv", index_col=0)

In [7]:
locations

,id,loc,correct_location
0,1.395719e+18,"Ile-de-France, France","Ile-Delaware, United States of America-France,..."
1,1.397449e+18,France,France
2,1.399296e+18,Hell,NaN
3,1.402200e+18,Belgique,Belgique
4,1.402205e+18,"Grenoble, France","Grenoble, France"
...,...,...,...
58487,1.513767e+18,Earth,NaN
58488,1.513767e+18,เกาหลี,เกาหลี
58489,1.513767e+18,Baltimore,Baltimore
58490,1.513768e+18,Plague Island,Plague Island


In [8]:
!pip install geopy

In [9]:
import pandas as pd
locations["geopy_output"] = pd.Series()
locations

C:\Users\hayadi\AppData\Local\Temp\ipykernel_19252\449326583.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  locations["geopy_output"] = pd.Series()


,id,loc,correct_location,geopy_output
0,1.395719e+18,"Ile-de-France, France","Ile-Delaware, United States of America-France,...",NaN
1,1.397449e+18,France,France,NaN
2,1.399296e+18,Hell,NaN,NaN
3,1.402200e+18,Belgique,Belgique,NaN
4,1.402205e+18,"Grenoble, France","Grenoble, France",NaN
...,...,...,...,...
58487,1.513767e+18,Earth,NaN,NaN
58488,1.513767e+18,เกาหลี,เกาหลี,NaN
58489,1.513767e+18,Baltimore,Baltimore,NaN
58490,1.513768e+18,Plague Island,Plague Island,NaN


In [40]:
import math

59

In [17]:
locations_sample = locations[1000:3000]

In [20]:
#Use geopy to get place context of each user location we precised
from tqdm import tqdm
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import math
import time 

geolocator = Nominatim(user_agent="tweet_geolocation")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

batch_size = 1000
for i in range(math.ceil(len(locations_sample)/batch_size)):
    print(i)
    try:
        locations_sample = locations_sample[i*batch_size : (i+1)*batch_size]
        print(len(locations_sample))
        locations_sample["geopy_output"] = locations_sample["correct_location"].apply(lambda x: geocode(x,  language='en'))
        locations_sample.to_csv('locations_sample_' + str(i+1) + '_with_geopy.csv')
    except Exception as e:
        print(e)
#locations["geopy_output"] = locations["correct_location"].apply(lambda x: geocode(x,  language='en'))


0
1000


RateLimiter caught an error, retrying (0/2 tries). Called with (*('The shady side of the street.',), **{'language': 'en'}).
Traceback (most recent call last):
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 1374, in getresponse
    response.begin()
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1"

RateLimiter swallowed an error after 2 retries. Called with (*('The shady side of the street.',), **{'language': 'en'}).
Traceback (most recent call last):
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 1374, in getresponse
    response.begin()
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
 

RateLimiter caught an error, retrying (1/2 tries). Called with (*('The shady side of the street.',), **{'language': 'en'}).
Traceback (most recent call last):
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 1374, in getresponse
    response.begin()
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1"

RateLimiter caught an error, retrying (0/2 tries). Called with (*('The shady side of the street.',), **{'language': 'en'}).
Traceback (most recent call last):
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 1374, in getresponse
    response.begin()
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1"

...

RateLimiter caught an error, retrying (1/2 tries). Called with (*('Istria, Laos mia patria',), **{'language': 'en'}).
Traceback (most recent call last):
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\util\connection.py", line 95, in create_connection
    raise err
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    sock.connect(sa)
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Pyt

RateLimiter caught an error, retrying (0/2 tries). Called with (*('England, United Kingdom',), **{'language': 'en'}).
Traceback (most recent call last):
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\util\connection.py", line 95, in create_connection
    raise err
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    sock.connect(sa)
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Pyt

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Batti Village The Gambia ',), **{'language': 'en'}).
Traceback (most recent call last):
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 1374, in getresponse
    response.begin()
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  

RateLimiter swallowed an error after 2 retries. Called with (*('Batti Village The Gambia ',), **{'language': 'en'}).
Traceback (most recent call last):
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 1374, in getresponse
    response.begin()
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  Fil

RateLimiter caught an error, retrying (1/2 tries). Called with (*('Batti Village The Gambia ',), **{'language': 'en'}).
Traceback (most recent call last):
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 1374, in getresponse
    response.begin()
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "C:\Users\hayadi\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  

In [ ]:
pd.read_csv('locations_sample_0_with_geopy.csv', index_col = 0)